# Laboratório do Pêndulo

## Modelo PHET

Simulação do Pêndulo utilizando o modelo do "Pendulum Lab" do PHET.
Fonte: https://github.com/phetsims/pendulum-lab/blob/master/doc/model.md

Notation:

$θ$: Angle of the pendulum from the resting position, in radians.

$ω$: Angular velocity of the pendulum, $ω = θ'$

$v$: Velocity of the pendulum, $v = ω \cdot L = θ' L$

$m$: Mass of the pendulum, in kg.

$g$: Acceleration due to gravity.

$L$: Length of the pendulum.

$c$: Drag coefficient

Given $θ$ (the angle of the pendulum from the resting position, in radians), there are three forces that we consider:

- Gravity ($mg$)

- Tension (centripetal force, $mv^2/L$)

- Drag ($c v^2 m^{2/3}$) http://fy.chalmers.se/~f7xiz/TIF081C/pendulum.pdf notes squared velocity due to Reynolds number surface area increases by $m^{2/3}$, and '$c$' is the drag coefficient

The net tangential force $= -m g sin( θ ) - c v^2 m^{2/3}$
$= -m g sin( θ ) - c L^2(θ')^2 m^{2/3}$ -- expansion of $v = θ' L$
= $m * a_tangential$
= $m * L * θ''$

Rearranging results in the differential equation: $θ'' + ( c L (θ')^2/ m^{1/3} )  + (g/L) sin(θ) = 0$

Which is then transformed into a system of differential equations and integrated with 4th order Runge Kutta (Scipy).

Energy:

Kinetic energy: $(1/2) m v^2 = 0.5 m (L ω)^2$

Potential energy: Relative to the resting point of the mass, so our 'height' is $L (1 - cos(θ))$, with a resulting energy of $m g L (1 - cos(θ))$.

Thermal energy: total - kinetic - potential.

The 'step' button will move forward 10 milliseconds (0.01s).

The Period Timer will time one period trace for a selected pendulum.

The Return button will reset the position of the pendulum, but will not change any settings.

Maximum Pendulum values:

Length ( $0.10m < L < 1.00m$ )
Mass ( $0.10kg < M < 1.50kg$ )
Gravity ( $0.00m/ss < g < 25.00m/ss$)

### Pêndulo simples

In [118]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate


In [119]:
def pendmodelphet(t,y,c,L,m,g):
    theta, omega = y
    dydt = [omega,
            -c*L*omega*np.abs(omega)/m**(1/3)-g*np.sin(theta)/L]
    return dydt

In [120]:
def integ(g,L0,m,tsim,theta0,c):
    theta0=theta0*np.pi/180
    #Integração numérica das equações diferenciais
    sol=integrate.solve_ivp(pendmodelphet, [0.0,tsim], [theta0,0.0], args=(c,L0,m,g),max_step=0.01)
    #Calculo da energia mecânica e energia dissipada ao longo do tempo
    v=sol.y[1,:]*L0 # velocidade linear na extremidade do pêndulo
    h=(1.-np.abs(np.cos(sol.y[0,:])))*L0 # Altura do pendulo com relação ao ponto mais baixo

    Emec=0.5*m*v**2+m*g*h # energia mecanica
    Ediss=m*g*h[0]-Emec # energia dissipada
    
    if c==0:
        Emec[:]=m*g*h[0]
        Ediss[:]=0
   
    plt.rcParams['figure.dpi'] = 100.0  
    plt.plot(sol.t, sol.y[0,:], label="L = " + str(L0) + 'm') 
    plt.title('Theta vs time')
    plt.xlabel('t (s)')
    plt.ylabel(r'$\theta$ (rad)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, sol.y[1,:], label="L = " + str(L0) + 'm') 
    plt.title('Angular velocity vs time')
    plt.xlabel('t (s)')
    plt.ylabel(r'$\omega$ (rad/s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, Emec, label="L = " + str(L0) + 'm') 
    plt.title('Energy vs time')
    plt.xlabel('$t$ (s)')
    plt.ylabel('$E_m$ (J)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, Ediss, label="L = " + str(L0) + 'm') 
    plt.title('Thermal energy vs time')
    plt.xlabel('$t$ (s)')
    plt.ylabel('$E_{diss}$ (J)')
    plt.grid(True)
    plt.legend()
    plt.show()

In [121]:
pendulo = interactive(integ, g = widgets.FloatSlider(min=0.0,max=30.,step=0.1,value=9.8,description='gravity ($m/s^2$)'), 
                          L0 = widgets.FloatSlider(min=0.2,max=1.,step=0.01,value=0.5,description='lenght (m)'),
                          m = widgets.FloatSlider(min=0.5,max=10.,step=0.1,value=1.,description='mass (kg)'),   
                          tsim = widgets.FloatSlider(min=2.,max=40.,step=1.,value=20.,description='time simul.'),
                          theta0 = widgets.FloatSlider(min=1.,max=90.,step=1.,value=5.,description=r'$\theta_0$'),
                          c = widgets.FloatSlider(min=0,max=10.,step=0.1,value=0,description='drag resist.'))

In [122]:
display(pendulo)

interactive(children=(FloatSlider(value=9.8, description='gravity ($m/s^2$)', max=30.0), FloatSlider(value=0.5…

### Dois pêndulos simples

In [123]:
def integ2(L1,L2,tsim,theta0,c):
    theta0=theta0*np.pi/180
    g=9.8
    m=1
    #Integração numérica das equações diferenciais
    sol=integrate.solve_ivp(pendmodelphet, [0.0,tsim], [theta0,0.0], args=(c,L1,m,g),max_step=0.01)
    sol2=integrate.solve_ivp(pendmodelphet, [0.0,tsim], [theta0,0.0], args=(c,L2,m,g),max_step=0.01)
    #Calculo da energia mecânica e energia dissipada ao longo do tempo
    v=sol.y[1,:]*L1 # velocidade linear na extremidade do pêndulo
    h=(1.-np.abs(np.cos(sol.y[0,:])))*L1 # Altura do pendulo com relação ao ponto mais baixo
    Emec=0.5*m*v**2+m*g*h # energia mecanica
    Ediss=m*g*h[0]-Emec # energia dissipada
    
    v2=sol2.y[1,:]*L2 # velocidade linear na extremidade do pêndulo
    h2=(1.-np.abs(np.cos(sol2.y[0,:])))*L2 # Altura do pendulo com relação ao ponto mais baixo
    Emec2=0.5*m*v2**2+m*g*h2 # energia mecanica
    Ediss2=m*g*h2[0]-Emec2 # energia dissipada
    
    if c==0:
        Emec[:]=m*g*h[0]
        Emec2[:]=m*g*h2[0]
        Ediss[:]=0
        Ediss2[:]=0
    
    plt.plot(sol.t, sol.y[0,:], label="L = " + str(L1) + 'm') 
    plt.plot(sol2.t, sol2.y[0,:], label="L = " + str(L2) + 'm')
    plt.title('Theta vs time')
    plt.xlabel('t (s)')
    plt.ylabel(r'$\theta$ (rad)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, sol.y[1,:], label="L = " + str(L1) + 'm')
    plt.plot(sol2.t, sol2.y[1,:], label="L = " + str(L2) + 'm') 
    plt.title('Angular velocity vs time')
    plt.xlabel('t (s)')
    plt.ylabel(r'$\omega$ (rad/s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, Emec, label="L = " + str(L1) + 'm')
    plt.plot(sol2.t, Emec2, label="L = " + str(L2) + 'm') 
    plt.title('Energy vs time')
    plt.xlabel('t (s)')
    plt.ylabel('$E_m$ (J)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, Ediss, label="L = " + str(L1) + 'm') 
    plt.plot(sol2.t, Ediss2, label="L = " + str(L2) + 'm')
    plt.title('Thermal energy vs time')
    plt.xlabel('t (s)')
    plt.ylabel('$E_{diss}$ (J)')
    plt.grid(True)
    plt.legend()
    plt.show()

In [124]:
pendulo2 = interactive(integ2, L1 = widgets.FloatSlider(min=0.2,max=1.,step=0.01,value=0.2),
                          L2 = widgets.FloatSlider(min=0.2,max=1.,step=0.01,value=0.8),
                          tsim = widgets.FloatSlider(min=2.,max=100.,step=1.,value=60.),
                          theta0 = widgets.FloatSlider(min=1.,max=90.,step=1.,value=5.),
                          c = widgets.FloatSlider(min=0.0,max=10.,step=0.1,value=5.0))

In [125]:
display(pendulo2)

interactive(children=(FloatSlider(value=0.2, description='L1', max=1.0, min=0.2, step=0.01), FloatSlider(value…

## Modelo de Força de arrasto proporcional à velocidade

$θ$: Angle of the pendulum from the resting position, in radians.

$ω$: Angular velocity of the pendulum, $ω = θ'$

$v$: Velocity of the pendulum, $v = ω \cdot L = θ' L$

$m$: Mass of the pendulum, in kg.

$g$: Acceleration due to gravity.

$L$: Length of the pendulum.

$b$: Drag coefficient

Given $θ$ (the angle of the pendulum from the resting position, in radians), there are three forces that we consider:

- Gravity ($mg$)

- Tension (centripetal force, $mv^2/L$)

- Drag ($b v$)

The net tangential force $= -m g sin( θ ) - b v$
$= -m g sin( θ ) - b L θ'$ -- expansion of $v = θ' L$
= $m * a_tangential$
= $m * L * θ''$

Rearranging results in the differential equation: $θ'' + b θ'/m  + (g/L) sin(θ) = 0$

Which is then transformed into a system of differential equations and integrated with 4th order Runge Kutta (Scipy).

Energy:

Kinetic energy: $(1/2) m v^2 = 0.5 m (L ω)^2$

Potential energy: Relative to the resting point of the mass, so our 'height' is $L (1 - cos(θ))$, with a resulting energy of $m g L (1 - cos(θ))$.

Thermal energy: total - kinetic - potential.

The 'step' button will move forward 10 milliseconds (0.01s).

The Period Timer will time one period trace for a selected pendulum.

The Return button will reset the position of the pendulum, but will not change any settings.

Maximum Pendulum values:

Length ( $0.10m < L < 1.00m$ )
Mass ( $0.10kg < M < 1.50kg$ )
Gravity ( $0.00m/ss < g < 25.00m/ss$)

### Dois pêndulos simples

In [126]:
def pendmodelv(t,y,b,L,m,g):
    theta, omega = y
    dydt = [omega,
            -b/m*omega*np.abs(omega)-g*np.sin(theta)/L]
    return dydt

In [127]:
def integ3(L1,L2,tsim,theta0,b):
    theta0=theta0*np.pi/180
    g=9.8
    m=1
    #Integração numérica das equações diferenciais
    sol=integrate.solve_ivp(pendmodelv, [0.0,tsim], [theta0,0.0], args=(b,L1,m,g),max_step=0.01)
    sol2=integrate.solve_ivp(pendmodelv, [0.0,tsim], [theta0,0.0], args=(b,L2,m,g),max_step=0.01)
    #Calculo da energia mecânica e energia dissipada ao longo do tempo
    v=sol.y[1,:]*L1 # velocidade linear na extremidade do pêndulo
    h=(1.-np.abs(np.cos(sol.y[0,:])))*L1 # Altura do pendulo com relação ao ponto mais baixo
    Emec=0.5*m*v**2+m*g*h # energia mecanica
    Ediss=m*g*h[0]-Emec # energia dissipada
    
    v2=sol2.y[1,:]*L2 # velocidade linear na extremidade do pêndulo
    h2=(1.-np.abs(np.cos(sol2.y[0,:])))*L2 # Altura do pendulo com relação ao ponto mais baixo
    Emec2=0.5*m*v2**2+m*g*h2 # energia mecanica
    Ediss2=m*g*h2[0]-Emec2 # energia dissipada
    
    if b==0:
        Emec[:]=m*g*h[0]
        Emec2[:]=m*g*h2[0]
        Ediss[:]=0
        Ediss2[:]=0
    
    
    plt.rcParams['figure.dpi'] = 100.0    
    plt.plot(sol.t, sol.y[0,:], label="L = " + str(L1) + 'm') 
    plt.plot(sol2.t, sol2.y[0,:], label="L = " + str(L2) + 'm')
    plt.title('Theta vs time')
    plt.xlabel('t (s)')
    plt.ylabel(r'$\theta$ (rad)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, sol.y[1,:], label="L = " + str(L1) + 'm')
    plt.plot(sol2.t, sol2.y[1,:], label="L = " + str(L2) + 'm') 
    plt.title('Angular velocity vs time')
    plt.xlabel('t (s)')
    plt.ylabel(r'$\omega$ (rad/s)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, Emec, label="L = " + str(L1) + 'm')
    plt.plot(sol2.t, Emec2, label="L = " + str(L2) + 'm') 
    plt.title('Energy vs time')
    plt.xlabel('t (s)')
    plt.ylabel('$E_m$ (J)')
    plt.grid(True)
    plt.legend()
    plt.show()
    plt.plot(sol.t, Ediss, label="L = " + str(L1) + 'm') 
    plt.plot(sol2.t, Ediss2, label="L = " + str(L2) + 'm')
    plt.title('Thermal energy vs time')
    plt.xlabel('t (s)')
    plt.ylabel('$E_{diss}$ (J)')
    plt.grid(True)
    plt.legend()
    plt.show()

In [128]:
pendulo3 = interactive(integ3, L1 = widgets.FloatSlider(min=0.2,max=1.,step=0.01,value=0.2),
                          L2 = widgets.FloatSlider(min=0.2,max=1.,step=0.01,value=0.8),
                          tsim = widgets.FloatSlider(min=2.,max=100.,step=1.,value=60.),
                          theta0 = widgets.FloatSlider(min=1.,max=90.,step=1.,value=5.),
                          b = widgets.FloatSlider(min=0.0,max=10.,step=0.1,value=4.0))

In [129]:
display(pendulo3)

interactive(children=(FloatSlider(value=0.2, description='L1', max=1.0, min=0.2, step=0.01), FloatSlider(value…